<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/main/%D0%90%D0%9A_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install kagglehub


In [26]:
import pandas as pd
import kagglehub
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

csv_path  = kagglehub.dataset_download("shantanudhakadd/email-spam-detection-dataset-classification")
csv_file = os.path.join(csv_path, 'spam.csv')
df = pd.read_csv(csv_file, encoding='latin1')

Using Colab cache for faster access to the 'email-spam-detection-dataset-classification' dataset.


In [27]:
df = df[['v1', 'v2']].copy()
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(df.head())
print(df['label'].value_counts())

   label                                               text
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...
label
0    4825
1     747
Name: count, dtype: int64


In [28]:
import numpy as np

class LSH:
    def __init__(self, num_buckets, num_hash_functions, input_dim):
        self.num_buckets = num_buckets
        self.num_hash_functions = num_hash_functions
        self.input_dim = input_dim
        self.hash_functions = [self._generate_hash_function() for _ in range(num_hash_functions)]
        self.buckets = {i: [] for i in range(num_buckets)}

    def _generate_hash_function(self):
        return np.random.randn(self.input_dim)

    def hash(self, vector):
        hashes = []
        for hash_function in self.hash_functions:
            hash_value = np.dot(vector, hash_function)
            hash_value = hash_value / np.linalg.norm(hash_function)
            hash_value = hash_value.astype(int)
            hashes.append(hash_value)
        return hashes

    def add_vector(self, vector):
        hashes = self.hash(vector)
        for i, hash_value in enumerate(hashes):
            self.buckets[hash_value % self.num_buckets].append((i, vector))

    def query(self, query_vector):
        query_hashes = self.hash(query_vector)
        similar_items = []
        for hash_value in query_hashes:
            bucket = self.buckets[hash_value % self.num_buckets]
            for item in bucket:
                if np.array_equal(item[1], query_vector):
                    continue
                if not any(np.array_equal(item[1], sim[1]) for sim in similar_items):
                    similar_items.append(item)
        return similar_items


In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

vectorizer = CountVectorizer()

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['text'])

svd = TruncatedSVD(n_components=50)
X_dense = svd.fit_transform(X_tfidf)

In [64]:
print(X_dense.shape)
#X_small = X_dense
X_small = X_dense[:500]

(5572, 50)


In [65]:
lsh = LSH(num_buckets=10, num_hash_functions=5, input_dim=X_small.shape[1])
for vec in X_small:
    lsh.add_vector(vec)
query_vec = X_small[0]
similar_items = lsh.query(query_vec)
print("Схожі об'єкти:", len(similar_items))

Схожі об'єкти: 489


In [66]:
print(f"Запитний лист:\n{df['text']}\nLabel: {'Spam' if df['label'].iloc[0]==1 else 'Ham'}\n")

# Виведемо перші 10 схожих листів
for idx, (i, vec) in enumerate(similar_items[:10]):
    similarity = np.dot(query_vec, vec) / (np.linalg.norm(query_vec) * np.linalg.norm(vec))
    print(f"#{idx+1} | Label: {'Spam' if df['label'].iloc[i]==1 else 'Ham'} | Cosine similarity: {similarity:.2f}")
    print(f"Text: {df['text']}...")


Запитний лист:
0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object
Label: Ham

#1 | Label: Ham | Cosine similarity: 0.04
Text: 0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I d

In [75]:
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['text'])
X_dense = X.toarray()

# Додаємо вектори окремо
lsh = LSH(num_buckets=10, num_hash_functions=5, input_dim=X_dense.shape[1])
for vec in X_dense[:2000]:
    lsh.add_vector(vec)

# Робимо запит
#query_vec = X_dense[0]  # перший email
query_text = ["Do you want a free phone?"]
# Перетворюємо у TF-IDF вектор
query_vec = vectorizer.transform(query_text).toarray()[0]
similar_items = lsh.query(query_vec)

print('Схожих текстів: ', len(similar_items))


Схожих текстів:  1890


In [76]:
# Тепер можна вивести реальні тексти та cosine similarity
for idx, (hash_id, vec) in enumerate(similar_items[:10]):
    cos_sim = np.dot(query_vec, vec) / (np.linalg.norm(query_vec)*np.linalg.norm(vec))
    print(f"#{idx+1} | Cosine similarity: {cos_sim:.2f}")
    print(df['text'].iloc[idx])
    print("--------------------------------------------------")

#1 | Cosine similarity: 0.00
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
--------------------------------------------------
#2 | Cosine similarity: 0.00
Ok lar... Joking wif u oni...
--------------------------------------------------
#3 | Cosine similarity: 0.09
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
--------------------------------------------------
#4 | Cosine similarity: 0.00
U dun say so early hor... U c already then say...
--------------------------------------------------
#5 | Cosine similarity: 0.00
Nah I don't think he goes to usf, he lives around here though
--------------------------------------------------
#6 | Cosine similarity: 0.03
FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
------------